In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
import string

# NLTK verilerini indir
nltk.download('stopwords')

# 1. Veri Yükleme ve Ön İşleme
def load_and_preprocess(filepath):
    df = pd.read_csv(filepath)
    
    # Eksik verileri temizleme
    df = df.dropna(subset=['content', 'category'])
    
    # Türkçe stopwords ve stemmer
    stop_words = set(stopwords.words('turkish'))
    stemmer = SnowballStemmer('turkish')
    
    def preprocess_text(text):
        if not isinstance(text, str):
            return ""
        
        # Küçük harfe çevirme
        text = text.lower()
        
        # Noktalama işaretlerini kaldırma
        text = text.translate(str.maketrans('', '', string.punctuation))
        
        # Sayıları kaldırma
        text = re.sub(r'\d+', '', text)
        
        # Stopwords ve stemming
        tokens = text.split()
        tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
        
        return ' '.join(tokens)
    
    df['processed_data'] = df['content'].apply(preprocess_text)
    
    return df

# 2. Veri Setini Yükle
try:
    df = load_and_preprocess('Datas/data.csv')
    print("Veri başarıyla yüklendi. Örnek veriler:")
    print(df.head())
    
    # Kategori dağılımını kontrol et
    print("\nKategori Dağılımı:")
    print(df['category'].value_counts())
    
except FileNotFoundError:
    print("Hata: 'data.csv' dosyası bulunamadı!")
    exit()
except Exception as e:
    print(f"Beklenmeyen hata: {e}")
    exit()

# 3. Veriyi Hazırla
X = df['processed_data']
y = df['category']

# Kategorileri sayısallaştır
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Eğitim ve test setlerini ayır
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# 4. Model Pipeline'ı Oluştur
model = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=10000,
        ngram_range=(1, 2),
        min_df=3,
        max_df=0.9
    )),
    ('clf', LinearSVC(
        C=1.0,
        class_weight='balanced',
        max_iter=10000
    ))
])

# 5. Modeli Eğit
print("\nModel eğitiliyor...")
model.fit(X_train, y_train)

# 6. Modeli Değerlendir
y_pred = model.predict(X_test)

print("\nModel Performansı:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print("\nDetaylı Rapor:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# 7. Yeni Veriyle Tahmin Yapma Örneği
def predict_category(text):
    processed_text = preprocess_text(text)
    prediction = model.predict([processed_text])
    return label_encoder.inverse_transform(prediction)[0]

# Test örneği
test_text = "Bu bir örnek metin, kategorisini tahmin etmeye çalışacağız"
predicted_category = predict_category(test_text)
print(f"\nÖrnek Tahmin: '{test_text[:30]}...' -> Kategori: {predicted_category}")

# 8. Modeli Kaydet (Opsiyonel)
import joblib
joblib.dump(model, 'text_classifier_model.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')
print("\nModel ve label encoder kaydedildi.")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\uslan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Beklenmeyen hata: The language 'turkish' is not supported.


NameError: name 'df' is not defined

: 